#   초기 조건 설정

우리가 갖고 있는건 섹터별 단위 시간(=주)에 대한 gap    
    
-> 해당 값들을 % 값 통해 각 섹터에 적용시키고 다시 한 번 가중치 합이 1이 되게 조정 필요    
    
-> 일단 맨 처음 섹터별 비중(=초기 조건)을 설정해줘야 함    
    
-> 이로 인해 KRX 데이터 통해 초기조건 데이터를 갖고 옴

## KRX 자료 가져오기 & 전처리

### KRX 자료 수합 및 산업 분류

우리가 필요한 섹터는    
통신, 헬스케어, 항공, 반도체, 금융(은증보 종합), 자동차, IT    
KRX 분류체계상 다음과 같음
    
-> 통신, 제약+의료정밀기기, 운송창고 , 전기전자, 은행증권보험기타금융, 운송장비부품, 일반서비스

In [4]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as plt

In [5]:
first_date_list = ['20181216', '20190317', '20190623', '20200301', '20201206', 
             '20210214', '20211128', '20220213', '20220724', '20221127', 
             '20230716', '20230924']

sector_list = ['제약', '전기·전자', '의료·정밀기기', '운송장비·부품', '운송·창고', '통신', '은행', '증권', '보험', '기타금융', '일반서비스']

mapping = {
    '제약': '헬스케어',
    '의료·정밀기기': '헬스케어',
    '전기·전자': '반도체',
    '운송장비·부품': '자동차',
    '운송·창고': '항공',
    '통신': '통신',
    '은행': '금융',
    '증권': '금융',
    '보험': '금융',
    '기타금융': '금융',
    '일반서비스': 'IT'
}

for date in first_date_list:
    globals()[f"raw_{date}"] = pd.read_excel(f"raw_{date}.xlsx", engine='openpyxl')
    globals()[f"raw_{date}"].set_index("업종명", inplace=True)
    globals()[f"df_{date}"] = globals()[f"raw_{date}"].loc[sector_list]

    # 여기부터는 섹터 종합하기 
    globals()[f"df_{date}"].rename(index=mapping, inplace=True)
    globals()[f"df_{date}"] = globals()[f"df_{date}"].groupby(globals()[f"df_{date}"].index).sum()

    df = globals()[f"df_{date}"]
    df.index.name = None


    # 각 "_비중" 컬럼을 해당 원본 컬럼의 가중치로 재계산
    for col in df.columns:
        if col.endswith("비중"):
            # '시가총액_비중' → 원본은 '시가총액_금액'
            # '거래량_비중' → 원본은 '거래량_수량'
            # '거래대금_비중' → 원본은 '거래대금_금액'
            measure = col.split("_")[0]
            if measure == "시가총액":
                weight_col = "시가총액_금액"
            elif measure == "거래량":
                weight_col = "거래량_수량"
            elif measure == "거래대금":
                weight_col = "거래대금_금액"
            else:
                continue
            
            total_weight = df[weight_col].sum()
            # 각 행의 가중치 값에 대한 비중 재계산
            df[col] = (df[weight_col] / total_weight) * 100

for date in first_date_list:
    globals()[f"df_{date}"] = globals()[f"df_{date}"].sort_values(by='시가총액_비중', ascending=False)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no de

In [6]:
df_20181216

,회사수,종목수,상장주식수,시가총액_금액,시가총액_비중,거래량_수량,거래량_비중,거래대금_금액,거래대금_비중
반도체,56,66,10830349,361861833,36.835054,41957,23.648671,1076653,34.569348
금융,120,152,13020788,270522237,27.537310,30147,16.992075,586096,18.818465
IT,79,83,3481583,112643932,11.466380,23060,12.997554,287808,9.240986
자동차,54,59,3433636,97386542,9.913282,59237,33.388382,419414,13.466612
헬스케어,49,56,2239857,80866773,8.231683,11502,6.482995,598262,19.209093
통신,4,4,868192,38656006,3.934916,2391,1.347665,81654,2.621760
항공,23,25,1711302,20447092,2.081374,9124,5.142657,64586,2.073738


### 포트폴리오 초기조건 df로 변경

In [7]:
for date in first_date_list:
    globals()[f"first_pf_{date}"] = globals()[f"df_{date}"]['시가총액_비중'].to_frame().T
    df = globals()[f"first_pf_{date}"]
    df.index = [f'{date}']
    df.index.name = '초기조건일'
    df.index = pd.to_datetime(df.index)


In [8]:
first_pf_20190317

,반도체,금융,IT,자동차,헬스케어,통신,항공
초기조건일,,,,,,,
2019-03-17,38.507647,26.906808,11.644225,9.881973,7.670559,3.349129,2.039659


## 주별 섹터별 가중치 계산하기

### Weekly_gap 데이터 수합

In [ ]:
event_date_list = ['20190315', '20190701', '20191006', '20200616', '20210118', '20210503', '20220309', '20220527', '20221102', '20231105', '20231025', '20240103']

mapping = {
    '20190315': '미시(기업리스트)',
    '20190701': '거시(정치이벤트)',
    '20191006': '미시(기업리스크)',
    '20200616': '거시(지정학리스크)',
    '20210118': '미시(기업리스크)',
    '20210503': '미시(제도적변화)',
    '20220309': '거시(정치이벤트)',
    '20220527': '미시(제도적변화)',
    '20221102': '거시(지정학리스크)',
    '20231105': '미시(제도적변화)',
    '20231025': '거시(지정학리스크)',
    '20240103': '거시(정치이벤트)'
}



In [9]:
event_date_list = ['20190315', '20190701', '20191006', '20200616', '20210321', '20210519', '20220309', '20220527', '20221105', '20230306', '20231025', '20240103']

for event_date in event_date_list:
    globals()[f"weekly_gap_{event_date}"] = pd.read_excel(f"weekly_df_gap_{event_date}.xlsx", engine='openpyxl')
    df = globals()[f"weekly_gap_{event_date}"]
    df.set_index("Unnamed: 0", inplace=True)
    df.index.name = 'Date'


In [10]:
weekly_gap_20190315

,weekly_gap_tlcm,weekly_gap_healthcare,weekly_gap_aerospace,weekly_gap_semi,weekly_gap_fin,weekly_gap_automobile,weekly_gap_it
Date,,,,,,,
2018-12-23,0.000000e+00,4.270840e-03,-0.374656,-0.165418,0.247289,0.000000e+00,-0.034787
2018-12-30,0.000000e+00,3.726851e-18,-0.336186,-0.055185,0.253509,0.000000e+00,-0.040783
2019-01-13,0.000000e+00,1.807696e-17,-0.644359,-0.042166,0.003409,-3.132678e-17,-0.008583
2019-01-20,1.336948e-15,0.000000e+00,-0.586605,-0.050235,0.073561,-7.664460e-18,-0.025384
2019-01-27,0.000000e+00,-2.187492e-03,-0.469565,-0.042905,0.213678,0.000000e+00,-0.026000
2019-02-03,0.000000e+00,1.129807e-18,-0.595387,-0.045820,0.324071,-1.244015e-17,-0.012210
2019-02-17,0.000000e+00,-2.289543e-03,-0.595996,-0.071782,0.281579,-1.220526e-17,-0.062253
2019-02-24,0.000000e+00,-2.735818e-03,-0.895468,-0.100056,0.290001,-7.302543e-18,-0.076669
2019-03-03,1.727360e-15,-1.322096e-17,-0.923439,-0.111690,0.323081,-3.297238e-17,-0.069782


In [11]:
first_pf_20181216

,반도체,금융,IT,자동차,헬스케어,통신,항공
초기조건일,,,,,,,
2018-12-16,36.835054,27.53731,11.46638,9.913282,8.231683,3.934916,2.081374


### 초기조건 df 와 weekly_gap 합치고 섹터별 가중치 계산

df 합치기

In [12]:
for event_date in event_date_list:
    first_date = globals()[f"weekly_gap_{event_date}"].index[0] - pd.Timedelta(days=7)
    first_date = first_date.strftime("%Y%m%d")
    
    df_gap = globals()[f"weekly_gap_{event_date}"]
    df_initial = globals()[f"first_pf_{first_date}"]

    mapping = {
        "weekly_gap_tlcm": "통신",
        "weekly_gap_healthcare": "헬스케어",
        "weekly_gap_aerospace": "항공",
        "weekly_gap_semi": "반도체",
        "weekly_gap_fin": "금융",
        "weekly_gap_automobile": "자동차",
        "weekly_gap_it": "IT"
    }
    df_gap_renamed = df_gap.rename(columns=mapping)

    # --------------------------
    # 4. Calculate the cumulative sum of the weekly gaps
    # --------------------------
 #   df_gap_cumsum = df_gap_renamed.cumsum()

    # --------------------------
    # 5. Add the cumulative gap to the initial condition
    # --------------------------
    # The initial condition is on 2018-12-16. To compute new levels,
    # add the cumulative gap to the initial condition values.
 #   df_updated = df_gap_cumsum.add(df_initial.iloc[0], axis=1)

    # --------------------------
    # 6. Optionally, prepend the initial condition row to have a complete time series.
    # --------------------------
    df_total = pd.concat([df_initial, df_gap_renamed])
    df_total = df_total.sort_index()
    globals()[f"pf_and_weekly_gap_{event_date}"] = df_total

시간대별 섹터 가중치 계산하기    
    
계산방식: 
1. 섹터값 * (1+ 다음 행 값(=gap))

2. 각 행에 대해 수행 후 총합 100 되도록 정규화(섹터별 가중치이므로)

In [13]:
pf_and_weekly_gap_20190315

,반도체,금융,IT,자동차,헬스케어,통신,항공
2018-12-16,36.835054,27.537310,11.466380,9.913282e+00,8.231683e+00,3.934916e+00,2.081374
2018-12-23,-0.165418,0.247289,-0.034787,0.000000e+00,4.270840e-03,0.000000e+00,-0.374656
2018-12-30,-0.055185,0.253509,-0.040783,0.000000e+00,3.726851e-18,0.000000e+00,-0.336186
2019-01-13,-0.042166,0.003409,-0.008583,-3.132678e-17,1.807696e-17,0.000000e+00,-0.644359
2019-01-20,-0.050235,0.073561,-0.025384,-7.664460e-18,0.000000e+00,1.336948e-15,-0.586605
2019-01-27,-0.042905,0.213678,-0.026000,0.000000e+00,-2.187492e-03,0.000000e+00,-0.469565
2019-02-03,-0.045820,0.324071,-0.012210,-1.244015e-17,1.129807e-18,0.000000e+00,-0.595387
2019-02-17,-0.071782,0.281579,-0.062253,-1.220526e-17,-2.289543e-03,0.000000e+00,-0.595996
2019-02-24,-0.100056,0.290001,-0.076669,-7.302543e-18,-2.735818e-03,0.000000e+00,-0.895468
2019-03-03,-0.111690,0.323081,-0.069782,-3.297238e-17,-1.322096e-17,1.727360e-15,-0.923439


In [14]:
for event_date in event_date_list:
    df = globals()[f"pf_and_weekly_gap_{event_date}"]
    df_updated = df.copy()

    base = df.iloc[0]

    for i in range(1, len(df)):
        # Calculate the new values for the row by applying the rate to the base values
        computed = base * (1 + df.iloc[i])
        
        # Normalize the computed values so that the sum equals 100
        total = computed.sum()
        normalized = computed / total * 100
        
        # Update the row in df_updated
        df_updated.iloc[i] = normalized

    globals()[f"final_sector_weight_{event_date}"] = df_updated
    globals()[f"final_sector_weight_{event_date}"].to_excel(f"final_sector_weight_{event_date}.xlsx", index=True)

In [15]:
final_sector_weight_20190315

,반도체,금융,IT,자동차,헬스케어,통신,항공
2018-12-16,36.835054,27.537310,11.466380,9.913282,8.231683,3.934916,2.081374
2018-12-23,30.873725,34.494277,11.114966,9.955795,8.302291,3.951791,1.307156
2018-12-30,33.534429,33.260717,10.598055,9.552131,7.931794,3.791563,1.331310
2019-01-13,36.335178,28.456099,11.707344,10.209235,8.477433,4.052390,0.762321
2019-01-20,35.458629,29.963516,11.326728,10.047596,8.343213,3.988230,0.872088
2019-01-27,34.224406,32.444761,10.841894,9.623590,7.973651,3.819928,1.071771
2019-02-03,33.202585,34.443956,10.699693,9.364784,7.776227,3.717199,0.795555
2019-02-17,33.151129,34.217922,10.425544,9.611791,7.963061,3.815244,0.815309
2019-02-24,32.648395,34.986166,10.427221,9.763430,8.085071,3.875435,0.214282
2019-03-03,32.060336,35.698526,10.450892,9.713143,8.065494,3.855475,0.156134


In [16]:
for i in range(len(final_sector_weight_20190315)):
    print(final_sector_weight_20190315.iloc[i].sum())

100.0
100.0
100.0
100.00000000000001
100.00000000000001
100.00000000000003
100.00000000000001
100.00000000000001
100.0
100.00000000000001
100.00000000000001
100.0
100.0
99.99999999999997
100.00000000000001
100.00000000000003
99.99999999999996
100.0
100.0
99.99999999999999
100.0
100.00000000000001
99.99999999999999
99.99999999999999
100.0
